In [1]:
from tqdm.notebook import tqdm

In [5]:
import spacy
import neuralcoref
nlp = spacy.load('en_core_web_lg')
#neuralcoref.add_to_pipe(nlp,blacklist=False)

In [59]:
neuralcoref.add_to_pipe(nlp,blacklist=False)

In [6]:
import os
import sys
import random
import numpy as np
import pandas as pd

git_dir = r"C:/Users/Aditya/GitHub/isquash"
#git_dir = "/Users/adityachhabra/Github/isquash"
sys.path.append(git_dir)
sys.path.append(os.path.join(git_dir,'isquash'))
pd.set_option('display.max_columns',500)

In [7]:
from isquash import nlpmodel, preprocessing, evaluation,crossval

In [8]:
human_coded_df   = pd.read_csv("data/annotated.csv")
uncoded_df       = pd.read_csv("data/unannotated.csv")

In [9]:
path = r"C:/Users/Aditya/Github/NLPQualToolkit/datasets/aspirations_r2_r3_v5.csv"

In [10]:
df = pd.read_csv(path)

In [11]:
transcript_dict = {g:"\n".join(gdf[['Q_en','A_en']].apply(" ".join,axis=1).values) for g, gdf in df.groupby('uid')}

In [12]:
def get_corefs(doc):
    pronouns = ['he','she','i','her','him','his','you','your','me','myself','my','they']
    all_names = []
    
    for coreference in doc._.coref_clusters:
        mentions = [token.text for token in coreference.mentions]
        #if m.text.lower() not in ['he']
        mentions = list(set([m for m in mentions if m.lower() not in pronouns]))
        if len(mentions)>0:
            all_names.append(mentions)
    return all_names

In [76]:
def get_ents(doc):
    entities = []
    coref_clusters = doc._.coref_clusters
    for cluster in coref_clusters:
        for token in cluster.mentions:
            entities.extend([{'text':token.text,'type':token.label_} for token in  token.ents])    
    return entities

In [77]:
pbar = tqdm(total=len(transcript_dict))
ENTITIES = []
for uid, transcript in transcript_dict.items():
    doc = nlp(transcript)
    ENTITIES.extend(get_ents(doc))
    pbar.update(1)

  0%|          | 0/2407 [00:00<?, ?it/s]

In [78]:
data = pd.DataFrame(ENTITIES)

In [84]:
entity_df = pd.DataFrame(data.groupby(['text','type']).value_counts(),columns=['count']).reset_index()

In [171]:
entity_path = r"C:/Users/Aditya/Desktop/entity_data.xlsx"

In [194]:
df = pd.read_excel(entity_path)

In [195]:
df['type'].value_counts()

PERSON      3153
ORG          271
LOCATION     153
NORP          56
DATE           1
Name: type, dtype: int64

In [196]:
word_replacements = {k:"["+v+"]" for k,v in dict(df[['text','type']].values).items() }

In [197]:
doc = nlp(transcript)

In [198]:
import difflib

In [199]:
import re

In [200]:
def replace_contractions(text,contractions):
    """
    Replaces all contractions in strings
    """
    for c in contractions:
        text = re.sub(r"\b{}\b".format(c),contractions[c],text)        
    return text

In [204]:
word_reps = {}
for k in sorted(word_replacements, key=len, reverse=True):
    word_reps[k] = word_replacements[k]


In [214]:
transcript = human_coded_df['Q_en'].sample(1).values[0]
print(transcript)

Okay then. Is your son Ziaul Rahman 17 years old?


In [215]:
print(replace_contractions(transcript,word_reps))

Okay then. Is your son [PERSON] 17 years old?
